In [1]:
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm 

class_to_new_id = {
    'General trash': 0,
    'Paper': 1,
    'Paper pack': 2,
    'Metal': 3,
    'Glass': 4,
    'Plastic': 5,
    'Styrofoam': 6,
    'Plastic bag': 7,
    'Battery': 8,
    'Clothing': 9
}

In [2]:
# object detection csv 파일
original_df = pd.read_csv("/home/hwang/leem/level2-objectdetection-cv-18/2-stages/level1-imageclassification-cv-18/post-processing/WBF_ATSS.csv")
original_df.head()

,PredictionString,image_id
0,7 0.8395863175392151 215.65679931640625 50.585...,test/0000.jpg
1,4 0.6536312103271484 344.48748779296875 249.65...,test/0001.jpg
2,1 0.7134121656417847 424.9185485839844 269.755...,test/0002.jpg
3,9 0.5041786432266235 139.1294708251953 264.084...,test/0003.jpg
4,0 0.5496219396591187 426.969482421875 407.7453...,test/0004.jpg


In [12]:
# tool

# PredictionString을 6개 단위로 나누는 함수 정의
def split_prediction_string(prediction_string):
    # 공백으로 나누어 리스트로 변환
    split_values = prediction_string.split()
    
    # 6개 단위로 묶어서 반환 (class_id, confidence_score, x1, y1, width, height)
    return [split_values[i:i+6] for i in range(0, len(split_values), 6)]

def save_cropimage_testinfocsv(image_id, bbox_info_list, savecrop):

    image_path = os.path.join("/hdd1/lim_data/level2_dataset", image_id)  # 'your_image_folder'에 실제 이미지가 위치
    img = Image.open(image_path)  # 이미지 로드
    test_info_data = []

    # 각 bounding box에 대해 크롭 후 저장
    for idx, bbox_info in enumerate(bbox_info_list):
        class_id, confidence, x1, y1, width, height = map(float, bbox_info)
        x1, y1, width, height = int(x1), int(y1), int(width), int(height)

        if savecrop == True:
            # Bounding box로 이미지 크롭
            cropped_img = img.crop((x1, y1, x1 + width, y1 + height))

            # 크롭된 이미지 저장 (파일 이름에 image_id와 bbox 번호 포함)
            cropped_img_name = f"{os.path.splitext(os.path.basename(image_id))[0]}_crop_{idx}.jpg"
            cropped_img_path = os.path.join(output_dir, cropped_img_name)
            cropped_img.save(cropped_img_path)

        cropped_img_name = f"{os.path.splitext(os.path.basename(image_id))[0]}_crop_{idx}.jpg"
        test_info_data.append({
            'label': int(class_id),  # label에 class_id를 임시로 저장
            'image_path': f"crop_test/{cropped_img_name}",
            'label_id': int(class_id),
            'cs' : confidence
        })

    return test_info_data


def save_detection_split_csv(df, output_csv_path):
    # CSV에 저장할 데이터를 담을 리스트
    split_data = []
    
    # 각 row에 대해 bbox 정보를 저장
    for index, row in tqdm(df.iterrows(), total=len(df)):
        image_id = row['image_id']
        bbox_info_list = row['bbox_info']

        for idx, bbox_info in enumerate(bbox_info_list):
            class_id, confidence, x1, y1, width, height = map(float, bbox_info)

            split_data.append({
                'image_id': os.path.basename(image_id),  # image_id의 base 경로만 저장
                'idx' : idx,
                'class_id': int(class_id),
                'confidence': confidence,
                'x1': x1,
                'y1': y1,
                'width': width,
                'height': height
            })

    # DataFrame으로 변환 후 CSV로 저장
    split_df = pd.DataFrame(split_data)
    split_df.to_csv(output_csv_path, index=False)

    return split_df

def convert_to_prediction_string_format(cropped_df, output_csv_path):
    # 새로 변환할 데이터를 담을 리스트
    prediction_data = []
    
    # image_id를 기준으로 그룹화하여 PredictionString을 생성
    grouped = cropped_df.groupby('image_id')
    
    for image_id, group in grouped:
        # 각 이미지에 대한 bbox 정보들을 PredictionString으로 결합
        prediction_string = " ".join(
            group.apply(lambda row: f"{int(row['class_id'])} {row['confidence']} {int(row['x1'])} {int(row['y1'])} {int(row['width'])} {int(row['height'])}", axis=1)
        )
        
        prediction_data.append({
            'PredictionString': prediction_string,
            'image_id': image_id
            
        })
    
    # DataFrame으로 변환 후 CSV로 저장
    prediction_df = pd.DataFrame(prediction_data)
    prediction_df.to_csv(output_csv_path, index=False)

    return prediction_df

In [4]:
root_dir = "/hdd1/lim_data/level2_dataset/test" # test 데이터 경로

# image_id & bbox info split
detection_df = original_df.copy(deep=False)
detection_df['bbox_info'] = detection_df['PredictionString'].apply(split_prediction_string)
detection_df = detection_df[['image_id', 'bbox_info']]
detection_df

,image_id,bbox_info
0,test/0000.jpg,"[[7, 0.8395863175392151, 215.65679931640625, 5..."
1,test/0001.jpg,"[[4, 0.6536312103271484, 344.48748779296875, 2..."
2,test/0002.jpg,"[[1, 0.7134121656417847, 424.9185485839844, 26..."
3,test/0003.jpg,"[[9, 0.5041786432266235, 139.1294708251953, 26..."
4,test/0004.jpg,"[[0, 0.5496219396591187, 426.969482421875, 407..."
...,...,...
4866,test/4866.jpg,"[[1, 0.47516515851020813, 290.8522644042969, 3..."
4867,test/4867.jpg,"[[3, 0.7905104756355286, 416.3888854980469, 32..."
4868,test/4868.jpg,"[[7, 0.47506701946258545, 118.60590362548828, ..."
4869,test/4869.jpg,"[[0, 0.31414151191711426, 22.799488067626953, ..."


In [21]:
output_csv_path = "/home/hwang/leem/level2-objectdetection-cv-18/2-stages/classification/csv/WBF_ATSS_split.csv"
split_df = save_detection_split_csv(detection_df, output_csv_path)
split_df


100%|██████████| 4871/4871 [00:00<00:00, 6592.85it/s]


,image_id,idx,class_id,confidence,x1,y1,width,height
0,0000.jpg,0,7,0.839586,215.656799,50.585876,455.092651,471.122375
1,0000.jpg,1,7,0.793338,448.197571,602.263062,647.875061,875.052856
2,0000.jpg,2,1,0.773484,554.609985,104.805801,755.877747,360.105255
3,0000.jpg,3,7,0.771402,604.898438,517.767456,957.673035,1022.789124
4,0000.jpg,4,7,0.762092,390.148102,191.706436,611.041016,546.604614
...,...,...,...,...,...,...,...,...
298283,4870.jpg,17,0,0.008579,375.659882,205.108673,625.276917,843.585938
298284,4870.jpg,18,4,0.008458,0.000000,0.000000,695.528300,730.060200
298285,4870.jpg,19,3,0.007832,374.605650,202.530060,624.209600,842.183350
298286,4870.jpg,20,7,0.007221,630.246640,0.000000,699.982600,557.047900


In [6]:
# test image crop & save , return test_info_csv
test_info_data = []  

for index, row in tqdm(detection_df.iterrows(), total=len(detection_df)):
    test_info_csv = save_cropimage_testinfocsv(row['image_id'], row['bbox_info'], savecrop=False)
    test_info_data.extend(test_info_csv)

test_info_df = pd.DataFrame(test_info_data)
test_info_df

100%|██████████| 4871/4871 [00:01<00:00, 3684.11it/s]


,label,image_path,label_id,cs
0,7,crop_test/0000_crop_0.jpg,7,0.839586
1,7,crop_test/0000_crop_1.jpg,7,0.793338
2,1,crop_test/0000_crop_2.jpg,1,0.773484
3,7,crop_test/0000_crop_3.jpg,7,0.771402
4,7,crop_test/0000_crop_4.jpg,7,0.762092
...,...,...,...,...
298283,0,crop_test/4870_crop_17.jpg,0,0.008579
298284,4,crop_test/4870_crop_18.jpg,4,0.008458
298285,3,crop_test/4870_crop_19.jpg,3,0.007832
298286,7,crop_test/4870_crop_20.jpg,7,0.007221


In [7]:
# class_id를 label로 변환 및 저장 (classification 모델 input info)
test_info_df['label'] = test_info_df['label'].replace({v: k for k, v in class_to_new_id.items()})

save_name = "/home/hwang/leem/level2-objectdetection-cv-18/2-stages/classification/csv/WBF_ATSS_test.csv"
test_info_df.to_csv(save_name, index=False)
test_info_df

,label,image_path,label_id,cs
0,Plastic bag,crop_test/0000_crop_0.jpg,7,0.839586
1,Plastic bag,crop_test/0000_crop_1.jpg,7,0.793338
2,Paper,crop_test/0000_crop_2.jpg,1,0.773484
3,Plastic bag,crop_test/0000_crop_3.jpg,7,0.771402
4,Plastic bag,crop_test/0000_crop_4.jpg,7,0.762092
...,...,...,...,...
298283,General trash,crop_test/4870_crop_17.jpg,0,0.008579
298284,Glass,crop_test/4870_crop_18.jpg,4,0.008458
298285,Metal,crop_test/4870_crop_19.jpg,3,0.007832
298286,Plastic bag,crop_test/4870_crop_20.jpg,7,0.007221


In [22]:
# classification prediction 으로 치환

# 1. 100% classification
    # return classification_df
# 2. confidence score를 고려해서 치환 (ex. detection 모델에선 낮은데 classification에선 높은것)
classification_df = pd.read_csv("/home/hwang/leem/level2-objectdetection-cv-18/2-stages/classification/csv/100_classification_WBF_ATSS.csv")
classification_df

,label,image_path,label_id,cs
0,Plastic bag,crop_test/0000_crop_0.jpg,7,0.995768
1,Plastic bag,crop_test/0000_crop_1.jpg,7,0.999213
2,Paper,crop_test/0000_crop_2.jpg,1,0.804213
3,Plastic bag,crop_test/0000_crop_3.jpg,7,0.996355
4,Plastic bag,crop_test/0000_crop_4.jpg,7,0.966343
...,...,...,...,...
298283,Glass,crop_test/4870_crop_17.jpg,4,0.967927
298284,Glass,crop_test/4870_crop_18.jpg,4,0.845347
298285,Glass,crop_test/4870_crop_19.jpg,4,0.973971
298286,Styrofoam,crop_test/4870_crop_20.jpg,6,0.898370


In [23]:
split_df['class_id'] = classification_df['label_id']
split_df['confidence'] = classification_df['cs']
split_df

,image_id,idx,class_id,confidence,x1,y1,width,height
0,0000.jpg,0,7,0.995768,215.656799,50.585876,455.092651,471.122375
1,0000.jpg,1,7,0.999213,448.197571,602.263062,647.875061,875.052856
2,0000.jpg,2,1,0.804213,554.609985,104.805801,755.877747,360.105255
3,0000.jpg,3,7,0.996355,604.898438,517.767456,957.673035,1022.789124
4,0000.jpg,4,7,0.966343,390.148102,191.706436,611.041016,546.604614
...,...,...,...,...,...,...,...,...
298283,4870.jpg,17,4,0.967927,375.659882,205.108673,625.276917,843.585938
298284,4870.jpg,18,4,0.845347,0.000000,0.000000,695.528300,730.060200
298285,4870.jpg,19,4,0.973971,374.605650,202.530060,624.209600,842.183350
298286,4870.jpg,20,6,0.898370,630.246640,0.000000,699.982600,557.047900


In [24]:
# original form 으로 변환
output_csv_path = "/home/hwang/leem/level2-objectdetection-cv-18/2-stages/classification/csv/100_WBF_ATSS.csv"

# 분리된 데이터를 다시 PredictionString 형식으로 변환
prediction_df = convert_to_prediction_string_format(split_df, output_csv_path)

# 변환된 데이터 확인
prediction_df

,PredictionString,image_id
0,7 0.99576783 215 50 455 471 7 0.9992126 448 60...,0000.jpg
1,5 0.9892095 344 249 753 695 5 0.59612477 131 0...,0001.jpg
2,1 0.9939395 424 269 641 325 1 0.95517147 881 4...,0002.jpg
3,9 1.0 139 264 893 820 9 1.0 14 17 1001 866 9 0...,0003.jpg
4,1 0.96249855 426 407 656 574 1 0.9692694 230 2...,0004.jpg
...,...,...
4866,1 0.69565874 290 347 670 688 1 0.78729576 290 ...,4866.jpg
4867,1 0.67160463 416 322 780 704 0 0.44733974 622 ...,4867.jpg
4868,7 0.9652715 118 453 752 841 0 0.5938101 0 670 ...,4868.jpg
4869,0 0.9997187 22 239 1001 925 7 0.5754572 838 0 ...,4869.jpg


In [26]:
# 일치 여부 확인

import pandas as pd

def compare_csv_format(file1_path, file2_path):
    # 두 CSV 파일 로드
    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)
    
    # 1. 컬럼 이름과 순서 비교
    columns_file1 = df1.columns.tolist()
    columns_file2 = df2.columns.tolist()
    
    if columns_file1 != columns_file2:
        print("❌ 두 CSV 파일의 컬럼 이름과 순서가 다릅니다.")
        print(f"파일 1의 컬럼: {columns_file1}")
        print(f"파일 2의 컬럼: {columns_file2}")
        return
    else:
        print("✅ 두 CSV 파일의 컬럼 이름과 순서가 동일합니다.")
    
    # 2. 행(row)과 열(column) 개수 비교
    if df1.shape != df2.shape:
        print(f"❌ 두 CSV 파일의 크기가 다릅니다. (파일 1: {df1.shape}, 파일 2: {df2.shape})")
        return
    else:
        print(f"✅ 두 CSV 파일의 크기(행과 열 개수)가 동일합니다. (크기: {df1.shape})")
    
    # 3. 첫 번째 열의 문자열 형식 비교 (공백 개수 확인)
    first_column_file1 = df1.iloc[:, 0].astype(str)
    first_column_file2 = df2.iloc[:, 0].astype(str)
    
    mismatch_count = 0
    
    for i in range(len(first_column_file1)):
        space_count_file1 = first_column_file1[i].count(' ')
        space_count_file2 = first_column_file2[i].count(' ')
        
        if space_count_file1 != space_count_file2:
            print(f"❌ 파일 1과 파일 2의 첫 번째 열에서 공백 개수가 다릅니다. (행 {i + 1})")
            print(f"파일 1: '{first_column_file1[i]}' (공백 {space_count_file1}개)")
            print(f"파일 2: '{first_column_file2[i]}' (공백 {space_count_file2}개)")
            mismatch_count += 1
            
    if mismatch_count == 0:
        print("✅ 첫 번째 열의 문자열 형식(공백 개수)이 두 파일에서 동일합니다.")
    else:
        print(f"총 {mismatch_count}개의 행에서 공백 개수가 다릅니다.")
    
# 사용 예시
file1_path = "/home/hwang/leem/level2-objectdetection-cv-18/2-stages/classification/csv/100_WBF_ATSS.csv"
file2_path = "/home/hwang/leem/level2-objectdetection-cv-18/2-stages/level1-imageclassification-cv-18/post-processing/WBF_ATSS.csv"
compare_csv_format(file1_path, file2_path)

✅ 두 CSV 파일의 컬럼 이름과 순서가 동일합니다.
✅ 두 CSV 파일의 크기(행과 열 개수)가 동일합니다. (크기: (4871, 2))
✅ 첫 번째 열의 문자열 형식(공백 개수)이 두 파일에서 동일합니다.
